In [1]:
#Assumptions
# pick each url from file1 and compare it against all urls in file2. 

import os
import requests

# Making REST API call. In case of any exceptions like invalid url, timeout, api result not being json, we will return the result as null.
def makeAPICall(url):
    try:
        response = requests.get(url)
        json_result = response.json()
    except:
        json_result = null
    return json_result


# JsonResponse from different API could be in any order. Inorder to make the comparison easier, we will be ordering the json object.
# Json is key-value pair where the value could either be dictionary or list. We should recursively order the values as well.
def orderJson(json_obj):
    if isinstance(json_obj, dict):
        return sorted((k, orderJson(v)) for k, v in json_obj.items())
    if isinstance(json_obj, list):
        return sorted(orderJson(x) for x in json_obj)
    else:
        return json_obj
    

# Function to make APICall and store in cache.  
# If the result is already present in cache, we will not make API Call, thus making the code efficient 
# While storing in cache, we will be ordering the json response so as to make comparison easier in future. 
def getAPIResponse(url, result_cache):
    url = url.rstrip()
    if url in result_cache: 
        return result_cache[url]
    result = makeAPICall(url)
    result_cache[url] = orderJson(result)
    return result_cache[url] 

#creating dictionary for efficient storage of api response. This way we don't need to make api calls for the same url again. 
result_cache = {} 
    
#iterate over each url from file1 and compare against each other in file2
def main(file1_content, file2_content):
    for url1 in file1_content:
        response1 = getAPIResponse(url1, result_cache)
        for url2 in file2_content:
            response2 = getAPIResponse(url2, result_cache)
            if response1 and response2 and response1 == response2:
                print(url1, "equals", url2)
            else:  
                print(url1, "not equals", url2)
                

path = os.getcwd()
file1_path = os.path.join(path, "file1.txt")
file2_path = os.path.join(path, "file2.txt")

#read file1's content
try:
    f1 = open(file1_path)
    file1_content = f1.readlines()
    f1.close()
except:
    print(file1_path, "file doesn't exists")

#read file2's content
try:
    f2 = open(file2_path)
    file2_content = f2.readlines()
    f2.close()
except:
    print(file1_path, "file doesn't exists")
    
main(file1_content, file2_content)
    

http://api.plos.org/search?q=title:DNA equals http://api.plos.org/search?q=title:DNA
